In [ ]:
#hide
#default_exp tools.query
from nbdev.showdoc import show_doc
import os

max_count=10
while not os.path.exists('settings.ini'):
    os.chdir('..')
    max_count = max_count - 1
    if max_count <= 0:
        break

# Query

> Shows results for queried experiments.

In [ ]:
#export
import warnings
warnings.filterwarnings('ignore')

import argparse
import sys
sys.path.append('.')
from collections import namedtuple
from IPython.display import display
import pandas as pd

# hpsearch api
import hpsearch.utils.experiment_utils as ut


def query (pv = {}, pf = {}, pall=[], pexact=False, root= None, 
           metric=None, experiments=None, runs=None, op=None, stats=['mean'], 
           results=0, other_parameters=False):
    
    
    result_query = ut.query(folder_experiments=root, suffix_results='_'+metric, experiments = experiments,
                        classes=runs, parameters_fixed=pf, parameters_variable=pv, parameters_all = pall, exact_match=pexact, 
                        ascending=op=='min', stats=stats, min_results=results, query_other_parameters=other_parameters)
    
    if not other_parameters:
        result_query = result_query[1]
        result_query = result_query['stats']

    return result_query 

def do_query_and_show (pall=[], best=None, compact=0, exact=False, experiments=None, pf={}, last=None, 
                       metric=None, op=None, other_parameters=False, input_range=None, results=0, 
                       root=None, round=2, runs=None, show=False, stats=['mean'], pv={},
                       sort=None, display_all_columns=False, col_width=None):
    
    from hpsearch.config.hpconfig import get_default_operations
    default_operations = get_default_operations ()
    if root is None:
        root = default_operations.get('root', 'results')
    if metric is None:
        metric = default_operations.get('metric', 'accuracy')
    if op is None:
        op = default_operations.get('op', 'min')
        
        
    df = query (pv=pv, pf=pf, pall=pall, pexact=exact, root=root, 
               metric=metric, experiments=experiments, runs=runs, op=op, stats=stats, 
               results=results, other_parameters=other_parameters)
    df = ut.replace_with_default_values (df)
    if sort is not None:
        assert sort in df.columns, f'sort must be a column in dataframe ({df.columns})'
        df = df.sort_values(by=sort, ascending=(op=='min'))
    if experiments is None:
        experiments = []
    if last is not None:
        experiments += range(df.index.max()-last+1, df.index.max()+1)
    if best is not None:
        experiments += list(df.index[:best])
    if input_range is not None:
        assert len(input_range) == 2
        experiments += range(input_range[0], input_range[1])
    if len(experiments) > 0: 
        df = df.loc[[x for x in df.index if x in experiments]]
    
    if col_width is not None:
        pd.set_option('max_colwidth', col_width)
    
    if (round is not None) and (round != 0):
        df[stats] = df[stats].round(round)
    if display_all_columns:
        display (df)
    
    print (f'experiments: {list(df.index)}')
    print (f'min experiment #: {df.index.min()}, max experiment #: {df.index.max()}')

    print ('result of query:')
    _, df2 = ut.get_parameters_unique(df)
    #df2.index.name = 'experiment #'
    if compact > 0:
        prev_cols = df2.columns.copy()
        df2, dict_rename = ut.compact_parameters (df2, compact)
        for k, kor in zip(df2.columns, prev_cols):
            print (f'{k} => {kor}')
    display (df2)
            
    if show:
        import hpsearch.visualization.plot_visdom as pv
        pv.plot_multiple_histories(df.index, root_folder=root,metrics=metric, parameters=None)
    return df2

In [ ]:
#export

def parse_args(args):
    default_always = ''

    parser = argparse.ArgumentParser(description='show metrics in visdom browser')
    # Datasets
    parser.add_argument('-m','--metric', type=str, default=None, help="metrics scores")
    parser.add_argument('--stats', type=str, nargs='+', default=['mean'],  help="statistics for multiple runs")
    parser.add_argument('--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('-r','--root', type=str, default=None)
    parser.add_argument('-v', type=str, default='{}', help='variable parameters')
    parser.add_argument('-f', type=str, default='{}', help='fixed parameters')
    parser.add_argument('-a', type=str, default='[]', help='all parameters')
    parser.add_argument('-e', '--exact', action= "store_true", help='exact match') 
    parser.add_argument('--last', type=int, default=None, help='include these last experiments') 
    parser.add_argument('--best', type=int, default=None, help='include these best experiments')
    parser.add_argument('--range', type=int, nargs='+', default=None, help='include this range of experiments')
    parser.add_argument('-c', '--compact', type=int, default=0, help='compact parameters to this number of characters') 
    parser.add_argument('--results', type=int, default=0, help='min number of results to consider') 
    parser.add_argument('-s', '--show', action= "store_true")
    parser.add_argument('--other', action= "store_true")
    parser.add_argument('--always', type=str, default = default_always)
    parser.add_argument('--op', default=None, type=str)
    parser.add_argument('--round', default=2, type=int, help='round scores to this number of digits')
    parser.add_argument('--runs', default=None, type=int, nargs='+', help='query restricted to run number provided')
    parser.add_argument('--sort', default=None, type=str)
    parser.add_argument('--width', default=None, type=int, help='max column width')
    pars = parser.parse_args(args)

    pars.v = eval(pars.v)
    pars.f = eval(pars.f)
    pars.a = eval(pars.a)
    pars.always = eval('dict(%s)' %pars.always)
    pars.f.update(pars.always)

    print (f'dictionary of query terms={pars.f}')
    
    return pars

def parse_arguments_and_query (args):
    
    pars = parse_args(args)

    do_query_and_show (pall=pars.a, best = pars.best, compact = pars.compact, exact=pars.exact, experiments=pars.experiments,
                       pf = pars.f, last=pars.last, metric = pars.metric, op = pars.op, other_parameters=pars.other,
                       input_range=pars.range, results=pars.results, root= pars.root, round=pars.round, runs = pars.runs,
                       show=pars.show, stats=pars.stats, pv = pars.v, sort=pars.sort, col_width=pars.width)

def main():
    parse_arguments_and_query (sys.argv[1:])

## Test

### Produce data

In [ ]:
#collapse_output
from hpsearch.examples.dummy_experiment_manager import remove_previous_experiments, run_multiple_experiments

remove_previous_experiments()
run_multiple_experiments(nruns=5, noise=0.1)

experiment script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line: 205
processing hyper-parameter 0 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7f8f82e79fd0>
running experiment 0
run number: 0

parameters:
	epochs=5,
	noise=0.1,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.00093841552734375
0 - validation_accuracy: 0.3159977302339135
0 - test_accuracy: 0.30377934742362456
finished experiment 0
processing hyper-parameter 0 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 0
run number: 1

parameters:
	epochs=5,
	noise=0.1,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this 

using import method 1
using import method 1
registering
class2import not found, switching to original method
using import method 1
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
registering
current path /mnt/athena/hpsearch/nbs/tools not found in source path /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
registering
current path /mnt/athena/hpsearch/nbs/tools not found in source path /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
registering
current path /mnt/athena/hpsearch/nbs/tools not found in source path /mnt/athena/hpsearch/hpsearch/examples/dummy_exp

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0009467601776123047
4 - validation_accuracy: 0.15127658275260397
4 - test_accuracy: 0.26896148908049444
finished experiment 0
processing hyper-parameter 1 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 0

parameters:
	epochs=5,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0008966922760009766
0 - validation_accuracy: 0.4742469973565302
0 - test_accuracy: 0.5978321548297305
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 1

parameters:
	epochs=5,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experi

fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
registering
current path /mnt/athena/hpsearch/nbs/tools not found in source path /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
registering
current path /mnt/athena/hpsearch/nbs/tools not found in source path /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
registering
current path /mnt/athena/hpsearch/nbs/tools not found in source path /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py
fitting model with 5 e

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0009152889251708984
3 - validation_accuracy: 0.4718068479141926
3 - test_accuracy: 0.5960880738093096
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 4

parameters:
	epochs=5,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0008866786956787109
4 - validation_accuracy: 0.40107428489646624
4 - test_accuracy: 0.5577856433636894
finished experiment 1
processing hyper-parameter 2 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 2
run number: 0

parameters:
	epochs=5,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experim

...

fitting model with 30 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
epoch 5: accuracy: 0.28
epoch 6: accuracy: 0.31000000000000005
epoch 7: accuracy: 0.3400000000000001
epoch 8: accuracy: 0.3700000000000001
epoch 9: accuracy: 0.40000000000000013
epoch 10: accuracy: 0.43000000000000016
epoch 11: accuracy: 0.4600000000000002
epoch 12: accuracy: 0.4900000000000002
epoch 13: accuracy: 0.5200000000000002
epoch 14: accuracy: 0.5500000000000003
epoch 15: accuracy: 0.5800000000000003
epoch 16: accuracy: 0.6100000000000003
epoch 17: accuracy: 0.6400000000000003
epoch 18: accuracy: 0.6700000000000004
epoch 19: accuracy: 0.7000000000000004
epoch 20: accuracy: 0.6700000000000004
epoch 21: accuracy: 0.6400000000000003
epoch 22: accuracy: 0.6100000000000003
epoch 23: accuracy: 0.5800000000000003
epoch 24: accuracy: 0.5500000000000003
epoch 25: accuracy: 0.5200000000000002
epoch 26: accuracy: 0.4900000000000002
epoch 27:

running experiment 7
run number: 3

parameters:
	epochs=30,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.003320932388305664
3 - validation_accuracy: 0.6277505529137108
3 - test_accuracy: 0.4806300376606064
finished experiment 7
processing hyper-parameter 7 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 7
run number: 4

parameters:
	epochs=30,
	noise=0.1,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0034449100494384766
4 - validation_accuracy: 0.6706214186607043
4 - test_accuracy: 0.6663783415443467
finished experiment 7
processing hyper-parameter 8 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 0

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hp

fitting model with 30 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
epoch 5: accuracy: 0.4800000000000001
epoch 6: accuracy: 0.5100000000000001
epoch 7: accuracy: 0.5400000000000001
epoch 8: accuracy: 0.5700000000000002
epoch 9: accuracy: 0.6000000000000002
epoch 10: accuracy: 0.6300000000000002
epoch 11: accuracy: 0.6600000000000003
epoch 12: accuracy: 0.6900000000000003
epoch 13: accuracy: 0.7200000000000003
epoch 14: accuracy: 0.7500000000000003
epoch 15: accuracy: 0.7800000000000004
epoch 16: accuracy: 0.8100000000000004
epoch 17: accuracy: 0.8400000000000004
epoch 18: accuracy: 0.8700000000000004
epoch 19: accuracy: 0.9000000000000005
epoch 20: accuracy: 0.8700000000000004
epoch 21: accuracy: 0.8400000000000004
epoch 22: accuracy: 0.8100000000000004
epoch 23: accuracy: 0.7800000000000004
epoch 24: accuracy: 0.7500000000000003
epoch 25: accuracy: 0.720000000

running experiment 8
run number: 2

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0034637451171875
2 - validation_accuracy: 0.988041976580603
2 - test_accuracy: 0.6459860477549018
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 3 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 3

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 205
time spent on this experiment: 0.0036301612854003906
3 - validation_accuracy: 0.9338155833811113
3 - test_accuracy: 0.9325193173752978
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 4

parameters:
	epochs=30,
	noise=0.1,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsea

fitting model with 30 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy: 0.8700000000000002
epoch 9: accuracy: 0.9000000000000002
epoch 10: accuracy: 0.9300000000000003
epoch 11: accuracy: 0.9600000000000003
epoch 12: accuracy: 0.9900000000000003
epoch 13: accuracy: 1.0200000000000002
epoch 14: accuracy: 1.0500000000000003
epoch 15: accuracy: 1.0800000000000003
epoch 16: accuracy: 1.1100000000000003
epoch 17: accuracy: 1.1400000000000003
epoch 18: accuracy: 1.1700000000000004
epoch 19: accuracy: 1.2000000000000004
epoch 20: accuracy: 1.1700000000000004
epoch 21: accuracy: 1.1400000000000003
epoch 22: accuracy: 1.1100000000000003
epoch 23: accuracy: 1.0800000000000003
epoch 24: accuracy: 1.0500000000000003
epoch 25: accuracy: 1.020000000000

### Run simple query

#### 1.- Run query without any condition, retrieving all the data

In [ ]:
arg_string = []
parse_arguments_and_query (arg_string)

dictionary of query terms={}
total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 8, 4, 2, 7, 3, 1, 6, 0]
min experiment #: 0, max experiment #: 8
result of query:


,epochs,offset,mean
5,15.0,0.6,0.95
8,30.0,0.6,0.87
4,15.0,0.3,0.80
2,5.0,0.6,0.78
7,30.0,0.3,0.62
3,15.0,0.1,0.50
1,5.0,0.3,0.48
6,30.0,0.1,0.42
0,5.0,0.1,0.28


In [ ]:
do_query_and_show ();

total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 8, 4, 2, 7, 3, 1, 6, 0]
min experiment #: 0, max experiment #: 8
result of query:


,epochs,offset,mean
5,15.0,0.6,0.95
8,30.0,0.6,0.87
4,15.0,0.3,0.80
2,5.0,0.6,0.78
7,30.0,0.3,0.62
3,15.0,0.1,0.50
1,5.0,0.3,0.48
6,30.0,0.1,0.42
0,5.0,0.1,0.28


#### Change the metric that we want to show

In [ ]:
arg_string = ['--metric', 'test_accuracy']
parse_arguments_and_query (arg_string)

dictionary of query terms={}
total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 2, 8, 4, 1, 7, 3, 0, 6]
min experiment #: 0, max experiment #: 8
result of query:


,epochs,offset,mean
5,15.0,0.6,0.97
2,5.0,0.6,0.89
8,30.0,0.6,0.81
4,15.0,0.3,0.58
1,5.0,0.3,0.56
7,30.0,0.3,0.55
3,15.0,0.1,0.43
0,5.0,0.1,0.36
6,30.0,0.1,0.34


In [ ]:
do_query_and_show(metric='test_accuracy');

total data examined: 9 experiments with at least 5 runs done for each one
experiments: [5, 2, 8, 4, 1, 7, 3, 0, 6]
min experiment #: 0, max experiment #: 8
result of query:


,epochs,offset,mean
5,15.0,0.6,0.97
2,5.0,0.6,0.89
8,30.0,0.6,0.81
4,15.0,0.3,0.58
1,5.0,0.3,0.56
7,30.0,0.3,0.55
3,15.0,0.1,0.43
0,5.0,0.1,0.36
6,30.0,0.1,0.34


#### 2.- Run query with conditions

In [ ]:
arg_string = ['--metric', 'validation_accuracy', 
              '--op', 'max', 
              '-f', 'dict(epochs=15)']
parse_arguments_and_query (arg_string)

dictionary of query terms={'epochs': 15}
total data examined: 3 experiments with at least 5 runs done for each one
experiments: [5, 4, 3]
min experiment #: 3, max experiment #: 5
result of query:


,offset,mean
5,0.6,0.95
4,0.3,0.80
3,0.1,0.50


In [ ]:
do_query_and_show (metric='validation_accuracy', op='max', pf={'epochs':15});

total data examined: 3 experiments with at least 5 runs done for each one
experiments: [5, 4, 3]
min experiment #: 3, max experiment #: 5
result of query:


,offset,mean
5,0.6,0.95
4,0.3,0.80
3,0.1,0.50


### Run query that sorts by maximum

In [ ]:
do_query_and_show (metric='validation_accuracy', op='max', sort='max', stats=['mean', 'min', 'max']);

In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)